In [2]:
import json
import requests

In [37]:
url = 'https://app.nanonets.com/api/v2/OCR/Model/08fe2528-e1e6-49c9-aff1-7aae3d13cba5/LabelFile/?async=false'

In [38]:
data = {'file': open('file path', 'rb')}

In [39]:
response = requests.post(url, auth=requests.auth.HTTPBasicAuth('API KEY', ''), files=data)
json_data = json.loads(response.text)

In [96]:
predictions = json_data.get("result", [])[0].get("prediction", [])
# print(response.text)

In [97]:
# Variables to store extracted data
merchant_name = ""
total_amount = 0.0  
tax_amount = 0.0    
date = ""
items = []

# Helper dictionary to organize items
item_details = {}

In [98]:
# Iterate through each prediction
for prediction in predictions:
    if prediction['label'] == 'Merchant_Name':
        merchant_name = prediction['ocr_text']
    elif prediction['label'] == 'Total_Amount':
        # Clean total
        clean_total_amount = prediction['ocr_text'].replace('$', '').replace(',', '') if prediction['ocr_text'] else '0.0'
        try:
            # print(clean_total_amount, type(clean_total_amount))  # comment out
            total_amount = float(clean_total_amount)  # truthy value else 0.0
            # print(total_amount, type(total_amount))  # comment out
        except ValueError:
            total_amount = 0.0  # float conversion error
    elif prediction['label'] == 'Tax_Amount':
        # Clean tax
        clean_tax_amount = prediction['ocr_text'].replace('$', '').replace(',', '') if prediction['ocr_text'] else '0.0'
        try:
            # print(clean_tax_amount, type(clean_tax_amount))  # comment out
            tax_amount = float(clean_tax_amount)  # truthy value else 0.0
            # print(tax_amount, type(tax_amount))  # comment out
        except ValueError: 
            tax_amount = 0.0  # float conversion error
    elif prediction['label'] == 'Date':
        date = prediction['ocr_text']
    elif prediction['label'] == 'table':
        for cell in prediction['cells']:
            row = cell['row']
            label = cell['label']
            text = cell['text']
            if row not in item_details:
                item_details[row] = {'Quantity': '1', 'Description': '', 'Line_Amount': ''}  # Default quantity to '1'
            if label in ['Quantity', 'Description', 'Line_Amount']:
                item_details[row][label] = text

# Convert item details to a list of structured dictionaries
item_list = []
for item in item_details.values():
    # Cleaning line amount
    clean_line_amount = item['Line_Amount'].replace('$', '').replace(',', '') if item['Line_Amount'] else '0.0'
    try:
        price = float(clean_line_amount)
    except ValueError:
        price = 0.0

    # Converting quantity to an integer
    try:
        quantity = int(item['Quantity']) if item['Quantity'] else 1
    except ValueError:
        quantity = 1  # Default to 1 if conversion fails

    item_list.append({
        'Name': item['Description'], 
        'Price': price,
        'Quantity': quantity
    })

In [99]:
final_output = {
    'Merchant Name': merchant_name,
    'Total Amount': total_amount,
    'Tax Amount': tax_amount,
    'Date': date,
    'Items': item_list
}

In [103]:
# Data type checks

print("Merchant Name type:", final_output['Merchant Name'], type(final_output['Merchant Name']))
print("Total Amount type:", final_output['Total Amount'], type(final_output['Total Amount']))
print("Tax Amount type:", final_output['Tax Amount'], type(final_output['Tax Amount']))
print("Date type:", final_output['Date'], type(final_output['Date']))

for i, item in enumerate(final_output['Items'], start=1):
    print(f"Item {i} - Name type: {type(item['Name'])}, Price type: {type(item['Price'])}, {item['Price']}, Quantity type: {type(item['Quantity'])}, {item['Quantity']}")

Merchant Name type: DIN TAI FUNG <class 'str'>
Total Amount type: 181.06 <class 'float'>
Tax Amount type: 14.26 <class 'float'>
Date type: 2023-07-18 <class 'str'>
Item 1 - Name type: <class 'str'>, Price type: <class 'float'>, 7.5, Quantity type: <class 'int'>, 1
Item 2 - Name type: <class 'str'>, Price type: <class 'float'>, 14.5, Quantity type: <class 'int'>, 1
Item 3 - Name type: <class 'str'>, Price type: <class 'float'>, 12.5, Quantity type: <class 'int'>, 1
Item 4 - Name type: <class 'str'>, Price type: <class 'float'>, 15.5, Quantity type: <class 'int'>, 1
Item 5 - Name type: <class 'str'>, Price type: <class 'float'>, 10.5, Quantity type: <class 'int'>, 1
Item 6 - Name type: <class 'str'>, Price type: <class 'float'>, 14.0, Quantity type: <class 'int'>, 1
Item 7 - Name type: <class 'str'>, Price type: <class 'float'>, 15.5, Quantity type: <class 'int'>, 1
Item 8 - Name type: <class 'str'>, Price type: <class 'float'>, 15.0, Quantity type: <class 'int'>, 1
Item 9 - Name type: <